# READING IMAGE REFERENCE DATA FOR POSE MAPPING
In this notebook, we evaluate the RDF GRAPHS in a session repository.
For every POINTCLOUDNODE resource, a POINTCLOUDNODE metadata class is created and its data is loaded. 
The following files will be imported and represented in memory
>pcdnodelist=PointCloudNode[]

>This codebase operates on the scan2bim2.yml environment (python 3.8)

In [13]:
# import APIs
import rdflib
from rdflib import Graph, plugin
from rdflib.serializer import Serializer #pip install rdflib-jsonld https://pypi.org/project/rdflib-jsonld/
from rdflib import Graph
from rdflib import URIRef, BNode, Literal
from rdflib.namespace import CSVW, DC, DCAT, DCTERMS, DOAP, FOAF, ODRL2, ORG, OWL, \
                           PROF, PROV, RDF, RDFS, SDO, SH, SKOS, SOSA, SSN, TIME, \
                           VOID, XMLNS, XSD

import os.path, time
import importlib
import PIL
import PIL.Image as PILimage
from PIL import ImageDraw, ImageFont, ImageEnhance
from PIL.ExifTags import TAGS, GPSTAGS
import xml.etree.ElementTree as ET
import numpy as np
import open3d as o3d

#IMPORT MODULES
import Algorithms.linkeddatatools as ld
import Classes.imagenode as im
import Algorithms.scan2bim as s2b

## 1. INITIALIZE SESSION

In [14]:
## INPUTS
project_path= "D:\\Data\\2018-06 Werfopvolging Academiestraat Gent"
session_path = project_path + "\\week 22\\PCD"
bim_folderpath = project_path + "\\BIM"
pcd_folderpath = session_path + "\\PCD"
img_folderpath = session_path + "\\IMG"
ortho_folderpath =session_path + "\\ORTHO"
mesh_folderpath = session_path + "\\MESH"

## 2. READ AND MERGE All THE RDF GRAPHS IN THE SESSION (save for feature.ttl files)

In [15]:
## read all graph files in a session and merge them
importlib.reload(ld)
importlib.reload(im)
importlib.reload(s2b)
all_session_file_paths=ld.getListOfFiles(session_path) 
rdf_graph_paths=[]

# collect all RDF graphs in the session
session_graph = Graph()
for file in all_session_file_paths:        
    if 'imgGraph' in file: 
        rdf_graph_paths.append(file)
        session_graph.parse(file)     
        print('imgGraph found with '+str(len(session_graph)) +' triples')

imgGraph found with 17170 triples


In [ ]:
#test
print(session_graph.serialize())

# 3. An ImageNode is created per image subject in the RDF Graph

In [16]:
importlib.reload(ld)
importlib.reload(im)
importlib.reload(s2b)

v4d3D=rdflib.Namespace('https://w3id.org/v4d/3D#')
session_graph.bind('v4d3D', v4d3D)

imgnodelist=[]

for s in session_graph.subjects(RDF.type, Literal(v4d3D.ImageNode)):
    # print(str(session_graph.value(subject=s,predicate=e57.recordCount) ))  
    imgnode=im.create_imagenode_from_rdf(session_graph=session_graph,s=s)
    imgnode.session_path=rdf_graph_paths[0] # this technically is not correct if there are more graphs involved
    imgnodelist.append(imgnode)

print (str(len(imgnodelist))+' ImageNodes succesfully created from the session!')

# alternative search options 
# find all subjects (s) of type (rdf:type) person (foaf:Person)
# for s, p, o in session_graph.triples((None, RDF.type, 'v4d3D.PointCloudNode')): #subjects(s), predicates (p) & objects (o) 
    # print(f"{s} is a PointCloudNode")

715 ImageNodes succesfully created from the session!


## 4. IMPORT IMAGES AS O3D.IMAGES
import o3d.image for each imagenode present

In [17]:
importlib.reload(ld)
importlib.reload(im)
importlib.reload(s2b)
counter=0
for idx,imgnode in enumerate(imgnodelist):
    if imgnode.img_path is not None:
        if imgnode.o3d_image is None:
            imgnode.o3d_image=o3d.io.read_image(imgnode.img_path)
        counter +=1
    
    elif imgnode.get_img_path() is True: # Return TRUE and set file path if file is found      
        if imgnode.o3d_image is None:
            imgnode.o3d_image=o3d.io.read_image(imgnode.img_path)            
        counter +=1

print (str(counter)+' of '+str(len(imgnodelist))+ ' ImageNodes have their o3d_image imported!')

AttributeError: type object 'ImageNode' has no attribute 'o3d_image'